In [20]:
import pandas as pd
from marginal.main import estimar_marginal
from thing import condicional


df = pd.read_csv("datasets/training_dataset1.csv")
df

,A,B,C
0,0,0,0
1,2,1,1
2,2,1,0
3,0,1,0
4,0,0,1
5,1,1,1
6,1,1,1
7,2,0,0
8,0,1,0
9,1,1,1


In [5]:
def foo(df,colNames, values):
    l = []
    for i in range(len(colNames)):
        l.append(df[colNames[i]] == values[i])
        
#     return df.loc()
#     print(l)
    
    x = l[0]
    for i in range(1,len(l)):
        x = x&l[i]
        
    return df.loc[x]

newDf = foo(df,["temperature"], ["hot"])
newDf

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,False,yes
3,overcast,hot,normal,False,yes
9,sunny,hot,high,False,no
10,sunny,hot,high,True,no


In [6]:
def findVariables(df, col):
    l = []
    for elem in df[col]:
    	if elem not in l:
    		l.append(elem)
    return l

x = findVariables(newDf, "outlook")
x

['overcast', 'sunny']

In [7]:
def getCardinality(df, col):
    return len(findVariables(df,col))

y = getCardinality(newDf, "outlook")
y

2

In [23]:
# Desde aqui importa <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

m = [[0,1,0],
     [0,0,1],
     [0,0,0]]

def getParents(g,df):
    fathers = []
    res = []
    colNames = list(df.columns)
    
    for col in range(len(g)):
        aux = []
        for row in range(len(g)):
            if(m[col][row] == 1):
                aux.append(colNames[row])
        fathers.append(aux)

    for i in range(len(g)):
        yield [colNames[i],fathers[i]]
    
    #return res

for i in getParents(m,df):
    print(i)

['A', ['B']]
['B', ['C']]
['C', []]


In [24]:
class rb:
    def __init__(self,g, df,a):
        self.dag = g
        self.margs = [] # => tienes que correr calc() para llenarlo
        self.conds = []
        self.alpha = a
        self.df = df
    
    def calc(self):
        for elem in getParents(self.dag,self.df):
            if(len(elem[1])== 0):
                self.margs.append([elem,estimar_marginal(df,elem[0],self.alpha)])
            else:
                self.conds.append([elem,condicional(df,elem[0],elem[1],self.alpha)])

In [49]:
def inferencia(df,query,colNames, values, rb):
    mgs = []
    cds = []
    accum = 1
    colNames = list(df.columns)
    
    
    for i in rb.conds:
        idxs = []
        for e in i[0]:
            if e in colNames:
                idxs.append(colNames.index(e))
            else:
                dictValues = i[1].values
                
                cds.append(max(dictValues))
        vals = []
        for idx in idxs:
            vals.append(values[idx])
        
        cds.append(i[1][vals])
    x
        
    for i in rb.margs: 
        idxs = []
        for e in i[0]:
            if e in colNames:
                idxs.append(colNames.index(e))
            else:
                dictValues = i.values
                mgs.append(max(dictValues))
        vals = []
        for idx in idxs:
            vals.append(values[idx])
            
            mgs.append(i[1][vals])
    
    for i in range(len(mgs)):
        accum *= mgs[i]
        
    for i in range(len(cds)):
        accum *= cds[i]
    return accum



In [33]:
x = rb(m,df,1)
x.calc()

10
6
10
4


In [50]:
print(inferencia(df,"A",["B"],["1"],x))

TypeError: 'builtin_function_or_method' object is not iterable

In [46]:
x.conds

[[['A', ['B']],
  {(0, 0): 0.5625,
   (0, 1): 0.28125,
   (2, 0): 0.375,
   (2, 1): 0.28125,
   (1, 0): 0.1875,
   (1, 1): 0.375}],
 [['B', ['C']],
  {(0, 0): 0.42857142857142855,
   (0, 1): 0.2857142857142857,
   (1, 0): 0.5714285714285714,
   (1, 1): 0.7142857142857143}]]